# Latencia
Tiempo que tarda un modelo en procesar una entrada y generar una prediccion

In [7]:
import os, time
import importlib , latency, pandas as pd # Importar importlib para recargar módulos
from dotenv import load_dotenv

dotenv_path = os.path.join(os.getcwd(), '..', '.env')
importlib.reload(latency)
from glob import glob
from latency import Latency

# Configuracion
config_file = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), os.getenv('CONFIG_SUBPATH'))
checkpoint_file = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), os.getenv('CHECKPOINT_SUBPATH'))
images_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('IMAGES_SUBPATH'))
labels_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('LABELS_SUBPATH'))
videos_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('VIDEOS_SUBPATH'))
evaluator = Latency(os.getenv('BASE_PATH'), images_path, labels_path, videos_path, config_file, checkpoint_file)
thresholds = [0.1, 0.5, 0.9]

Loads checkpoint by local backend from path: /Users/apple/Documents/pose_estimation/MMPosev1.1/configs/model_weights/hrnet_w32_coco_256x192-c78dce93_20200708.pth


# Calculo de latencia en un video especifico

In [8]:
# Obtener la lista de todas las imágenes en la carpeta
video = 'ejercicio01 - frente.mp4'
video_path = os.path.join(videos_path, 'uziel', 'ejercicio01', 'conLuz', video)
print(video_path)

results = []
for threshold in thresholds:
    results = evaluator.evaluate_video(
        videos_path=video_path, 
        threshold=threshold,
        results=results,
        frame_skip=5,
    )

/Users/apple/Documents/pose_estimation/dataset/videos/uziel/ejercicio01/conLuz/ejercicio01 - frente.mp4


Procesando (/Users/apple/Documents/pose_estimation/dataset/videos/uziel/ejercicio01/conLuz/ejercicio01 - frente.mp4) - (th:0.1): 100%|██████████| 175/175 [00:08<00:00, 20.46it/s]
Procesando (/Users/apple/Documents/pose_estimation/dataset/videos/uziel/ejercicio01/conLuz/ejercicio01 - frente.mp4) - (th:0.5): 100%|██████████| 175/175 [00:08<00:00, 20.80it/s]
Procesando (/Users/apple/Documents/pose_estimation/dataset/videos/uziel/ejercicio01/conLuz/ejercicio01 - frente.mp4) - (th:0.9): 100%|██████████| 175/175 [00:08<00:00, 20.43it/s]


# Calculo de latencia en videos dentro de una carpeta

In [10]:

folder_path = os.path.join(videos_path, 'uziel')
video_extensions = ['*.mp4', '*.avi', '*.mov']
video_files = []
results = []

for ext in video_extensions:
    video_files.extend(glob(os.path.join(folder_path, '**', ext), recursive=True))

for video_path in video_files:
    for threshold in thresholds:
        results = evaluator.evaluate_video(
            videos_path=video_path, 
            threshold=threshold, 
            results=results,
            frame_skip=5,
        )

Procesando (/Users/apple/Documents/pose_estimation/dataset/videos/uziel/ejercicio02/conLuz/ejercicio02 - izquierda obstaculo.mp4) - (th:0.1): 100%|██████████| 219/219 [00:11<00:00, 19.28it/s]
Procesando (/Users/apple/Documents/pose_estimation/dataset/videos/uziel/ejercicio02/conLuz/ejercicio02 - izquierda obstaculo.mp4) - (th:0.5):  14%|█▎        | 30/219 [00:01<00:10, 17.28it/s]


KeyboardInterrupt: 

# Generando XLS de los resultados de los videos

In [12]:
df_results = pd.DataFrame(results)
print(df_results.head().to_string)
output_excel_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), 'velocidad', 'latency', 'mmpose_video_latency_results.xlsx')
df_results.to_excel(output_excel_path, index=False, engine='openpyxl')

print(f"Los resultados se han guardado en {output_excel_path}")

<bound method DataFrame.to_string of                                    video video_resolution  threshold  \
0  ejercicio02 - izquierda obstaculo.mp4          480x864        0.1   
1  ejercicio02 - izquierda obstaculo.mp4          480x864        0.1   
2  ejercicio02 - izquierda obstaculo.mp4          480x864        0.1   
3  ejercicio02 - izquierda obstaculo.mp4          480x864        0.1   
4  ejercicio02 - izquierda obstaculo.mp4          480x864        0.1   

   video_fps  frame_skip  latency_ms  processed_frames  total_frames  
0  30.016722           5  270.565033                 1           219  
1  30.016722           5  249.023914                 2           219  
2  30.016722           5  234.563112                 3           219  
3  30.016722           5  233.150959                 4           219  
4  30.016722           5  228.711367                 5           219  >
Los resultados se han guardado en /Users/apple/Documents/pose_estimation/MMPosev1.1/velocidad/latency/m